# Fuel costs
PowerGenome uses fuel costs from EIA's Annual Energy Outlook scenarios. Model regions must be mapped to AEO fuel regions in the settings file. Users can then specify AEO scenarios for each fuel type. In addition to the parameters for generators, you should include these parameters in the settings:

- `eia_series_region_names` (map readable names to the codes for each Census division used in EIA open data)
- `aeo_fuel_region_map` (map model regions to Census division names from `eia_series_region_names`)
- `eia_series_fuel_names` (map readable names to the fuel codes used in EIA open data)
- `eia_aeo_year`
- `eia_series_scenario_names` (map readable names to the scenario codes used in EIA open data)
- `aeo_fuel_scenarios` (the AEO scenario used for each fuel type)
- `aeo_usd_year`
- `tech_fuel_maps` (the fuel name from `eia_series_fuel_names` used for each EIA technology - EIA to ATB technologies are mapped in `eia_atb_tech_map`)
- `ccs_fuel_map` (the name of fuels for CCS technologies)
- `ccs_capture_rate` (how much CO₂ from each fuel in `ccs_fuel_map` is captured)
- `ccs_disposal_cost`
- `carbon_tax` (optional)
- `fuel_emission_factors`

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import warnings

warnings.simplefilter("ignore")

In [3]:
from pathlib import Path

import pandas as pd
from powergenome.fuels import fuel_cost_table
from powergenome.generators import GeneratorClusters
from powergenome.util import (
    build_scenario_settings,
    init_pudl_connection,
    load_settings,
)
from powergenome.eia_opendata import fetch_fuel_prices

## Import settings
This assumes that the settings file is set up for multiple scenarios/planning periods. If you are using a settings file with only a single scenario/planning period, remove or comment out the line with `build_scenario_settings`.

In [4]:
pudl_engine, pudl_out, pg_engine = init_pudl_connection()
cwd = Path.cwd()

settings_path = (
    cwd.parent / "example_system" / "test_settings.yml"
)
settings = load_settings(settings_path)
settings["input_folder"] = settings_path.parent / settings["input_folder"]
scenario_definitions = pd.read_csv(
    settings["input_folder"] / settings["scenario_definitions_fn"]
)

scenario_settings = build_scenario_settings(settings, scenario_definitions)

## Create dataframe of generators (existing and new)
The dataframe will have a column `Fuel` that describes the fuel used by a resource in the format `<REGION>_<SCENARIO>_<FUEL>`.

In [5]:
gc = GeneratorClusters(pudl_engine, pudl_out, pg_engine, scenario_settings[2030]["p1"])
gen_clusters = gc.create_all_generators()

794.1999999999999  MW without lat/lon


Technology Conventional Hydroelectric changed capacity from 12699.700000000008 to 12704.200000000008


Creating gdf
['Solar Photovoltaic', 'Onshore Wind Turbine', 'Batteries', 'Biomass', 'Conventional Hydroelectric', 'Natural Gas Fired Combined Cycle', 'Other_peaker', 'All Other', 'Natural Gas Fired Combustion Turbine', 'Other Natural Gas']


No model tag values found for MinCapTag_2 ('MinCapTag_2')
No model tag values found for Reg_Max ('Reg_Max')
No model tag values found for Rsv_Max ('Rsv_Max')
Selected technology landbasedwind capacity in region CA_N less than minimum (8424.4314 < 25000 MW)
Selected technology landbasedwind capacity in region CA_S less than minimum (23639.682500000003 < 45000 MW)
No model tag values found for MinCapTag_2 ('MinCapTag_2')
No model tag values found for Reg_Max ('Reg_Max')
No model tag values found for Rsv_Max ('Rsv_Max')
Transmission investment costs are missing or zero for some resources and will not be included in the total investment costs.


In [6]:
gen_clusters[["region", "Resource", "Fuel"]]

,region,Resource,Fuel
0,CA_N,biomass,None
1,CA_N,conventional_hydroelectric,None
2,CA_N,geothermal,None
3,CA_N,hydroelectric_pumped_storage,None
4,CA_N,natural_gas_fired_combined_cycle,pacific_reference_naturalgas
...,...,...,...
61,WECC_AZ,utilitypv_losangeles_mid_0_2,None
62,WECC_AZ,utilitypv_losangeles_mid_0_2,None
63,WECC_AZ,utilitypv_losangeles_mid_0_2,None
64,CA_N,ev_load_shifting,None


## Build a fuel cost table
This table gives the cost (in the specified model year and AEO scenario) and CO₂ content for every fuel that is used by a generating resource.

In [7]:
fuels = fuel_cost_table(gc.fuel_prices, generators=gen_clusters, settings=gc.settings)

In [8]:
fuels

Fuel,None,pacific_reference_naturalgas,mountain_reference_coal,mountain_reference_naturalgas,mountain_reference_uranium,pacific_reference_naturalgas_ccs90,pacific_reference_uranium,pacific_reference_naturalgas_ccs100,mountain_reference_naturalgas_ccs90,mountain_reference_naturalgas_ccs100
Time_Index,,,,,,,,,,
0,0.0,0.05306,0.09552,0.05306,0.00,0.00531,0.00,0.00,0.00531,0.00
1,0.0,3.58000,1.40000,3.78000,0.66,4.06000,0.66,4.12,4.25000,4.31
2,0.0,3.58000,1.40000,3.78000,0.66,4.06000,0.66,4.12,4.25000,4.31
3,0.0,3.58000,1.40000,3.78000,0.66,4.06000,0.66,4.12,4.25000,4.31
4,0.0,3.58000,1.40000,3.78000,0.66,4.06000,0.66,4.12,4.25000,4.31
...,...,...,...,...,...,...,...,...,...,...
476,0.0,3.58000,1.40000,3.78000,0.66,4.06000,0.66,4.12,4.25000,4.31
477,0.0,3.58000,1.40000,3.78000,0.66,4.06000,0.66,4.12,4.25000,4.31
478,0.0,3.58000,1.40000,3.78000,0.66,4.06000,0.66,4.12,4.25000,4.31
